# Tuning Self-supervised Contrastive Learning

### Table of Content
1. [Dependencies](#import-and-set-up-dependencies)
    - [Tune Config](#tune-settings)
    - [Base Model Config](#base-model-configuration)
2. [Data](#prepare-datasets)
3. [Tuning Loop](#set-up-tuning-loop)
4. [Tune](#tune)
5. [Logging](#log-results)
6. [Test](#testing-using-tuned-model)

## Import and Set Up Dependencies

In [ ]:
#   Setup
##  Standard packages
import os
import sys
import time
import math
import logging
import numpy as np
from sklearn.linear_model import LogisticRegression

##  Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as vtransforms
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset, random_split
from torch.utils.data.sampler import WeightedRandomSampler
from torch import optim

##  SRC dataset, loader, and metrics
import src.data.dataset as ds
import src.data.dataloader as dl
import src.models as mdl
import src.utils.metric as customMetric
from src.utils.metric import calc_score

##  Self-supervised Contrastive Learning
from src.train import train_supcon, valid_supcon
from src.utils.supcontrast import TwoCropTransform, AverageMeter, SupConLoss
from src.utils.supcontrast import adjust_learning_rate, warmup_learning_rate
from src.utils.supcontrast import set_optimizer, save_model
from src.test import test_supcon

##  Tuning Packages
import ray
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search.hyperopt import HyperOptSearch

In [ ]:
# -------------------- Globals --------------------#
# Device Config
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {DEVICE} on {torch.cuda.get_device_name(0)} :D ")

# Dataset Config
TASK_IN = "Task_11"
MAX_LENGTH = 3.0
SR = 8000
HOP_LENGTH = 128
MAX_LENGTH_SAMPLES = int(MAX_LENGTH * SR / HOP_LENGTH)
INPUT_X_DIM = int(MAX_LENGTH * SR / HOP_LENGTH)
N_F_BIN = 64
N_FFT = 512
FEATURE = "mfcc"

# DataLoader Config
VAL_PERCENT = 0.2
BATCH_SIZE = 32

# Log Config
formatter = logging.Formatter("%(asctime)s:%(levelname)s:%(name)s:%(message)s")

#### Tune Settings

In [ ]:
TUNE_STRAT = ["score", "max"]
BASE_CONFIG = {}
TUNE_CONFIG = {
    "temperature": tune.uniform(0.1, 0.9),
    "lr": tune.loguniform(1e-4, 1e-2),
    
}
TUNE_MAX_EPOCH = 20
TUNE_GPU_PER_TRIAL = 1
TUNE_CPU_PER_TRIAL = 8
TUNE_SAMPLE_NUM = 10

### Base Model Configuration

In [ ]:
# -------------------- Define customized Argparse --------------------#
class modelSetting:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

    def print_args(self):
        argparse_dict = vars(self)
        for key, value in argparse_dict.items():
            print(f"{key}: {value}")

opt = modelSetting(
    # Dataset Config
    task_in = "Task_11", data_path = "SPRSound/", 
    batch_size = BATCH_SIZE, val_percent = 0.2,
    
    # Model Config
    model = "resnet18", embedding_size = 128, 
    head = "linear", ckpt = "best.pth", 

    # Train Config
    print_freq = 50, save_freq = 50, epochs = 50,

    # Optim Config
    optimizer = "SGD",
    learning_rate = 0.001, momentum = 0.9,
    lr_decay_rate = 0.1, lr_decay_epochs = "70,80,90",
    weight_decay = 1e-4, dropout = 0.25,

    # SupCon Config
    temperature = 0.5, method = "SupCon",

    # Other Config
    cosine = True, warm = False, verbose = False,
)

iterations = opt.lr_decay_epochs.split(",")
opt.lr_decay_epochs = list([])
for it in iterations:
    opt.lr_decay_epochs.append(int(it))

# warm-up for large-batch training,
if opt.batch_size > 256:
    opt.warm = True
if opt.warm:
    opt.warmup_from = 0.01
    opt.warm_epochs = 10
    if opt.cosine:
        eta_min = opt.learning_rate * (opt.lr_decay_rate ** 3)
        opt.warmup_to = eta_min + (opt.learning_rate - eta_min) * (
                1 + math.cos(math.pi * opt.warm_epochs / opt.epochs)) / 2
    else:
        opt.warmup_to = opt.learning_rate

# set the path according to the environment
opt.model_path = "./temp/SupCon-Notes/{}_models".format(opt.task_in)
opt.model_name = "{}_{}{}_{}{}_hop{}_{}_lr{}_temp{}_drop{}_val{}".format(
    opt.model, 
    FEATURE, 
    N_F_BIN, 
    opt.head,
    opt.embedding_size,
    HOP_LENGTH, 
    opt.optimizer,
    opt.learning_rate,
    opt.temperature,
    opt.dropout,
    opt.val_percent,
)
opt.save_folder = os.path.join(opt.model_path, opt.model_name)
if not os.path.isdir(opt.save_folder):
    os.makedirs(opt.save_folder)

opt.print_args()

In [ ]:
# -------------------- User-defined functions --------------------#
def setupLogger(name, logPath, level=logging.INFO):
    handler = logging.FileHandler(logPath)
    handler.setFormatter(formatter)
    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.addHandler(handler)
    return logger

## Set Globals
log_path = "logs/HyperTune.log"
if not os.path.exists(log_path):
    open(log_path, "a").close()
logger = setupLogger("TuneSupConLogger", log_path)
main_task = int(TASK_IN[-2])
sub_task = int(TASK_IN[-1])
data_path = "SPRSound"
num_classes = len(dl.classes[TASK_IN])

In [ ]:
data_dict={
    "train":[
        os.path.join(data_path, "train_wav"), 
        os.path.join(data_path, "train_json")
    ],
    "intra_test":[
        os.path.join(data_path, "test_wav"), 
        os.path.join(data_path, "test_json/intra_test_json")
    ],
    "inter_test":[
        os.path.join(data_path, "test_wav"),
        os.path.join(data_path, "test_json/inter_test_json")
    ],
}

[Home](#table-of-content)

## Prepare Datasets

In [ ]:
## Preapare transformation and generate dataset
train_transform = vtransforms.Compose([
    vtransforms.RandomHorizontalFlip(),
    vtransforms.RandomCrop(size=(N_F_BIN, MAX_LENGTH_SAMPLES), padding=0, pad_if_needed=True),
])

trainDataset, intra_testDataset, inter_testDataset = ds.genDatasets(
    task=main_task, 
    data_dict=data_dict,
    resample=None,
    feature=FEATURE,
    pre_emph=False,
    pos_norm="zscore",
    n_mfcc=N_F_BIN,
    hop_length=HOP_LENGTH,
    n_fft=N_FFT,
)

In [ ]:
supcon_loader = dl.trainValLoader(
    trainDataset,
    sub_task,
    valid_size=VAL_PERCENT,
    batch_size=BATCH_SIZE,
    collate_fn=lambda batch: dl.supcon_collate(
        batch, TASK_IN, sub_task, transform=TwoCropTransform(train_transform)
    ),
    train_sampler="balanced",
    val_sampler="balanced",
)

print("\n\nGenerating Dataloader for Train Dataset...")
dataloader = dl.trainValLoader(
    trainDataset,
    sub_task,
    valid_size=VAL_PERCENT,
    batch_size=BATCH_SIZE,
    collate_fn=lambda batch: dl.custom_collate(
        batch, MAX_LENGTH_SAMPLES, TASK_IN, sub_task
    ),
    train_sampler="balanced",
    val_sampler="balanced",
)

print("\n\nGenerating Dataloader for Intra Dataset...")
intra_testloader = dl.testLoader(
    intra_testDataset,
    batch_size=BATCH_SIZE,
    collate_fn=lambda batch: dl.custom_collate(
        batch, MAX_LENGTH_SAMPLES, TASK_IN, sub_task
    ),
    shuffle_in=False,
)

print("\nGenerating Dataloader for Inter Dataset...")
inter_testloader = dl.testLoader(
    inter_testDataset,
    batch_size=BATCH_SIZE,
    collate_fn=lambda batch: dl.custom_collate(
        batch, MAX_LENGTH_SAMPLES, TASK_IN, sub_task
    ),
    shuffle_in=False,
)

[Home](#table-of-content)

## Set up Tuning-Loop

In [ ]:
def set_model(opt):
    model = mdl.SupConResNet(
        name=opt.model, 
        head=opt.head,
        feat_dim=opt.embedding_size,
        dropout=opt.dropout,
    )
    criterion = SupConLoss(temperature=opt.temperature)

    if torch.cuda.is_available():
        if torch.cuda.device_count() > 1:
            model.encoder = torch.nn.DataParallel(model.encoder)
        model = model.cuda()
        criterion = criterion.cuda()
        cudnn.benchmark = True
    
    return model, criterion

In [ ]:
def trainingLoop(config, opt, supcon_loader, dataloader):
    opt.learning_rate = config["lr"]
    opt.temperature = config["temperature"]
    model, criterion = set_model(opt)
    optimizer = set_optimizer(opt, model)
    print("\n\nTraining...")
    print("Running for {} epochs...".format(opt.epochs))
    best_loss = 0
    best_epoch = 1
    # training routine
    for epoch in range(1, opt.epochs + 1):
        adjust_learning_rate(opt, optimizer, epoch)

        # train for one epoch
        time1 = time.time()
        train_loss = train_supcon(supcon_loader["train"], model, criterion, optimizer, epoch, opt)
        valid_loss = valid_supcon(supcon_loader["val"], model, criterion, opt)
        time2 = time.time()
        print("epoch {}, total time {:.2f}, train loss: {:.2f}, valid loss: {:.2f}".format(epoch, time2 - time1, train_loss, 1/valid_loss))

        if valid_loss > best_loss:
            best_loss = valid_loss
            best_model = model
            best_optimizer = optimizer
            best_epoch = epoch
    print("\n\nTesting..")
    best_model.eval()
    targets = []
    embeddings = torch.zeros((0, opt.embedding_size), dtype=torch.float32)
    for data, label, _ in dataloader["train"]:
        data = data.to(DEVICE)
        embedding = best_model(data)
        targets.extend(label.detach().cpu().tolist())
        embeddings = torch.cat((embeddings, embedding.detach().cpu()), dim=0)
    x_embed = np.array(embeddings)
    y = np.array(targets)

    # Create a logistic regression classifier
    classifier = LogisticRegression()
    classifier.fit(x_embed, y)
    predictions = classifier.predict(x_embed)

    print("\nResult for Train:")
    train_score, *_ = calc_score(y, predictions, verbose=True, task=int(opt.task_in[-2]))
    print("\nResult for Valid:")
    val_score = test_supcon(best_model, classifier, dataloader["val"], opt)
    # Here we save a checkpoint. It is automatically registered with
    # Ray Tune and can be accessed through `session.get_checkpoint()`
    # API in future iterations.
    os.makedirs("tuning_models", exist_ok=True)
    torch.save(
        (best_model.state_dict(), optimizer.state_dict()),
        "tuning_models/checkpoint.pt",
    )
    checkpoint = Checkpoint.from_directory("tuning_models")
    session.report(
        {
            "score": val_score,
        },
        checkpoint=checkpoint,
    )



In [ ]:
strat_target, strat_mode = TUNE_STRAT
scheduler = ASHAScheduler(max_t=TUNE_MAX_EPOCH, grace_period=1, reduction_factor=2)
algo = HyperOptSearch(metric=strat_target, mode=strat_mode)
tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(
            trainingLoop, opt=opt, supcon_loader=supcon_loader, dataloader=dataloader
        ),
        resources={"cpu": TUNE_CPU_PER_TRIAL, "gpu": TUNE_GPU_PER_TRIAL},
    ),
    tune_config=tune.TuneConfig(
        metric=strat_target,
        mode=strat_mode,
        scheduler=scheduler,
        num_samples=TUNE_SAMPLE_NUM,
        search_alg=algo,
    ),
    param_space=TUNE_CONFIG,
)

[Home](#table-of-content)

## Tune

In [ ]:
results = tuner.fit()

[Home](#table-of-content)

## Log Results

In [ ]:
best_result = results.get_best_result(strat_target, strat_mode)
best_val_score = best_result.metrics["score"]
# best_val_loss = best_result.metrics["loss"]
# best_val_accu = best_result.metrics["accuracy"]
print("Best trial config: {}".format(best_result.config))
# print("Best trial final validation loss: {}".format(best_val_loss))
# print("Best trial final validation accuracy: {}".format(best_val_accu))
print("Best trial final validation score: {}".format(best_val_score))

[Home](#table-of-content)

## Testing using Tuned Model

In [ ]:
## Test the best Network
### Set the opt based on best result ----- Edit this part for different variables
opt.temperature = best_result.config["temperature"]
opt.learning_rate = best_result.config["lr"]
### --------------------------------------------------
test_network = mdl.SupConResNet(
    name=opt.model, 
    head=opt.head,
    feat_dim=opt.embedding_size,
    dropout=opt.dropout,
).to(DEVICE)
best_chkpt = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")

model_state, _ = torch.load(best_chkpt)
test_network.load_state_dict(model_state)
test_network.eval()

targets = []
embeddings = torch.zeros((0, opt.embedding_size), dtype=torch.float32)
for data, label, _ in dataloader["train"]:
    data = data.to(DEVICE)
    embedding = test_network(data)
    targets.extend(label.detach().cpu().tolist())
    embeddings = torch.cat((embeddings, embedding.detach().cpu()), dim=0)

x_embed = np.array(embeddings)
y = np.array(targets)

# Create a logistic regression classifier
classifier = LogisticRegression()
classifier.fit(x_embed, y)
predictions = classifier.predict(x_embed)

with torch.no_grad():
    print("\nResult for Intra:")
    intra_score = test_supcon(test_network, classifier, intra_testloader, opt)
    print("\nResult for Inter:")
    inter_score = test_supcon(test_network, classifier, inter_testloader, opt)

In [ ]:
logMessage = (
    f"SupContrast with: {opt.model}, Task: {TASK_IN}, inter score: {inter_score:>0.3}, "
    f"intra score: {intra_score:>0.3}, val_score: {best_val_score:>0.3}, best trial config: {best_result.config}")
logger.info(logMessage)